In [1]:
from selenium import webdriver

import time
import numpy as np
import pandas as pd
import tabula

from bs4 import BeautifulSoup
import urllib.request as req

import requests
import os
import glob
import datetime
import re

import io

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage

# テキスト変換器の設定
rsrcmgr = PDFResourceManager()
outfp = io.StringIO()

device = TextConverter(
    rsrcmgr,
    outfp,
    laparams=LAParams(),
    imagewriter=None
)

In [2]:
select = glob.glob('men/stats/*.csv')
for i in range(len(select)):
    select[i] = select[i][10:-4]
# len(select)

In [642]:
# C51: 選手の基礎情報
# C69: ライブスコア
# C73: 試合結果
# C83: 試合スタッツ

# pdf_path = 'OG2020-_VVO_C51_VVOMTEAM6-------------FNL-000200--.pdf'
pdf = 'OG2020-_VVO_C73_VVOMTEAM6-------------FNL-000200--.pdf'
dfs = tabula.read_pdf(pdf, stream=True, pages='all')
print(len(dfs))

5


In [18]:
# テキスト変換器の設定
rsrcmgr = PDFResourceManager()
outfp = io.StringIO()

device = TextConverter(
    rsrcmgr,
    outfp,
    laparams=LAParams(),
    imagewriter=None
)

pdf = 'men/pdf/OG2020-_VVO_C83_VVOMTEAM6-------------FNL-000200--.pdf'
# pdf = 'men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------GPA-000100--.pdf'
# pdf = 'men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------QFNL000100--.pdf'
dfs = tabula.read_pdf(pdf, stream=True, pages='all')
sex = 'men'
print(len(dfs))

with open(pdf, "rb") as fp:
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)

    # 出力結果の表示
    outfp.seek(0)
    text = outfp.readlines()
    # print(text)
    gameList = []
    teamList = []
    for t in text:
    #     print(t)
        if 'vs' in t:
            team = t.replace('\n', '').replace(' ', '')
            teamList.append(team)
        if any(x in t for x in ('Round', 'final', 'Medal Match', 'Bronze')):
            game = t.replace('\n', '').replace(' ', '')
            gameList.append(game)
    # print(team, gameList[0])
    name = '{}/stats/{}-{}'.format(sex, gameList[0], teamList[0])
    team1 = teamList[0][:3]
    team2 = teamList[0][-3:]
    print(name)
for i in range(len(dfs)):
    # print(score, play)
    if i % 4 != 0:
        print(i)
        col = dfs[i].columns
        print(col[0])
        play = col[0].split()
        dfs[i].iloc[:, 1] = dfs[i].iloc[:, 1].str[-2:]
        for j in range(len(col)):
            if type(dfs[i].iloc[0, j]) == str:
                dfs[i].iloc[:, j] = dfs[i].iloc[:, j].str.replace('-', '')
        dfs[i] = dfs[i].drop(columns={col[0], col[-2]}).rename(columns={'Unnamed: 0':'In-Play', 'Unnamed: 1':'%', 'TotalRk No.':'No.', 'Avg.Rk No.':'No.'})
#         print(dfs[i].head())
        if i <= 8:
            csv_name = '{}-{}-{}.csv'.format(name, team1, col[0])
        else:
            csv_name = '{}-{}-{}.csv'.format(name, team2, col[0])
        print(csv_name)
        dfs[i].to_csv(csv_name, index=False)

16
men/stats/BronzeMedalMatch-ARGvsBRA
1
Scoring Eff. %Attack
men/stats/BronzeMedalMatch-ARGvsBRA-ARG-Scoring Eff. %Attack.csv
2
Scoring In-PlayBlock
men/stats/BronzeMedalMatch-ARGvsBRA-ARG-Scoring In-PlayBlock.csv
3
Scoring In-PlayService
men/stats/BronzeMedalMatch-ARGvsBRA-ARG-Scoring In-PlayService.csv
5
Non-Scoring In-PlayDig
men/stats/BronzeMedalMatch-ARGvsBRA-ARG-Non-Scoring In-PlayDig.csv
6
Non-Scoring In-PlaySet
men/stats/BronzeMedalMatch-ARGvsBRA-ARG-Non-Scoring In-PlaySet.csv
7
Non-Scoring Succ. %Reception
men/stats/BronzeMedalMatch-ARGvsBRA-ARG-Non-Scoring Succ. %Reception.csv
9
Scoring Eff. %Attack
men/stats/BronzeMedalMatch-ARGvsBRA-BRA-Scoring Eff. %Attack.csv
10
Scoring In-PlayBlock
men/stats/BronzeMedalMatch-ARGvsBRA-BRA-Scoring In-PlayBlock.csv
11
Scoring In-PlayService
men/stats/BronzeMedalMatch-ARGvsBRA-BRA-Scoring In-PlayService.csv
13
Non-Scoring In-PlayDig
men/stats/BronzeMedalMatch-ARGvsBRA-BRA-Non-Scoring In-PlayDig.csv
14
Non-Scoring In-PlaySet
men/stats/Bronze

In [11]:
print(teamList[0][:3])
print(teamList[0][-3:])

BRA
ROC


In [17]:
dfs[1]

,No.,Name,Points,Errors,In-Play,Attempts,%
0,7,CONTE Facundo,19.0,6,21,46,28.26
1,12,LIMA Bruno,11.0,7,7,25,16.00
2,13,PALACIOS Ezequiel,10.0,6,10,26,15.38
3,11,SOLE Sebastian,9.0,1,4,14,NaN
4,8,LOSER Agustin,7.0,NaN,5,12,NaN
5,15,de CECCO Luciano,1.0,NaN,2,3,NaN
6,2,PEREYRA Federico,1.0,NaN,,1,NaN
7,1,SANCHEZ Matias,NaN,,,,NaN
8,6,POGLAJEN Cristian,NaN,,,,NaN
9,9,DANANI Santiago,NaN,,,,NaN


In [20]:
sex = 'women'
pdfs = glob.glob('{}/pdf/*C83*.pdf'.format(sex))
for pdf in pdfs:
    print(pdf)
    teamList = ['(cid:']
    gameList = ['(cid:']
    dfs = tabula.read_pdf(pdf, stream=True, pages='all')
    #     print(len(dfs))
    while '(cid:' in teamList[0] or '(cid:' in gameList[0]:
        # テキスト変換器の設定
        rsrcmgr = PDFResourceManager()
        outfp = io.StringIO()

        device = TextConverter(
            rsrcmgr,
            outfp,
            laparams=LAParams(),
            imagewriter=None
        )
        
        with open(pdf, "rb") as fp:
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            for page in PDFPage.get_pages(fp):
                interpreter.process_page(page)

            # 出力結果の表示
            outfp.seek(0)
            text = outfp.readlines()
#             print(text[0:5])
            gameList = []
            teamList = []
            for t in text:
            #     print(t)
                if 'vs' in t:
                    team = t.replace('\n', '').replace(' ', '')
                    teamList.append(team)
    #             if 'Round' in t or 'final' in t or 'Medal Match' in t or 'Bronze' in t:
                if any(x in t for x in ('Round', 'final', 'Medal Match', 'Bronze')):
                    game = t.replace('\n', '').replace(' ', '')
                    gameList.append(game)
#             print(teamList[0], gameList[0])
        name = '{}/stats/{}-{}'.format(sex, gameList[0], teamList[0])
        print(name)
        team1 = teamList[0][:3]
        team2 = teamList[0][-3:]
    for i in range(len(dfs)):
        # print(score, play)
        if i % 4 != 0:
#             print(i)
            col = dfs[i].columns
#             print(col[0])
            play = col[0].split()
            dfs[i].iloc[:, 1] = dfs[i].iloc[:, 1].str[-2:]
            for j in range(len(col)):
                if type(dfs[i].iloc[0, j]) == str:
                    dfs[i].iloc[:, j] = dfs[i].iloc[:, j].str.replace('-', '')
            dfs[i] = dfs[i].drop(columns={col[0], col[-2]}).rename(columns={'Unnamed: 0':'In-Play', 'Unnamed: 1':'%', 'TotalRk No.':'No.', 'Avg.Rk No.':'No.'})
#             print(dfs[i].head(3))
            if i <= 8:
                csv_name = '{}-{}-{}.csv'.format(name, team1, col[0])
            else:
                csv_name = '{}-{}-{}.csv'.format(name, team2, col[0])
#             print(csv_name)
            dfs[i].to_csv(csv_name, index=False)

women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------FNL-000100--.pdf
women/stats/GoldMedalMatch-BRAvsUSA
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------FNL-000200--.pdf
women/stats/BronzeMedalMatch-KORvsSRB
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------GPA-000100--.pdf
women/stats/PreliminaryRound-PoolA-SRBvsDOM
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------GPA-000200--.pdf
women/stats/PreliminaryRound-PoolA-JPNvsKEN
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------GPA-000300--.pdf
women/stats/PreliminaryRound-PoolA-BRAvsKOR
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------GPA-000400--.pdf
women/stats/PreliminaryRound-PoolA-JPNvsSRB
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------GPA-000500--.pdf
women/stats/PreliminaryRound-PoolA-BRAvsDOM
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------GPA-000600--.pdf
women/stats/PreliminaryRound-PoolA-KORvsKEN
women/pdf\OG2020-_VVO_C83_VVOWTEAM6-------------GPA-000700--.pdf
women/stats/PreliminaryRound-PoolA-KORvsDOM
women/pdf\OG2020-_VVO_C83_VVOWTEA

In [178]:
dfs[1]

,Name,Points,Errors,In-Play,%
0,in,21.0,5,14,40.00
1,an,12.0,5,14,22.58
2,or,10.0,11,7,NaN
3,as,7.0,NaN,4,NaN
4,my,6.0,NaN,3,NaN
5,ne,1.0,NaN,1,NaN
6,ia,NaN,,,NaN
7,in,NaN,,,NaN
8,in,NaN,,,NaN
9,en,NaN,,,NaN


In [36]:
import PyPDF2
from PyPDF2.pdf import ContentStream
pdf

'men/pdf\\OG2020-_VVO_C83_VVOMTEAM6-------------SFNL000200--.pdf'

In [37]:
pdf2 = pdf
print(pdf2)

men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------SFNL000200--.pdf


In [41]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

input_path = pdf2
output_path = 'result.txt'

manager = PDFResourceManager()

with open(output_path, "wb") as output:
    with open(input_path, 'rb') as input:
        with TextConverter(manager, output, codec='utf-8', laparams=LAParams()) as conv:
            interpreter = PDFPageInterpreter(manager, conv)
            for page in PDFPage.get_pages(input):
                interpreter.process_page(page)

In [171]:
print(pdf)

men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------GPA-000100--.pdf


In [167]:
team

'(cid:41)RAvsROC'

In [173]:

# テキスト変換器の設定
rsrcmgr = PDFResourceManager()
outfp = io.StringIO()

device = TextConverter(
    rsrcmgr,
    outfp,
    laparams=LAParams(),
    imagewriter=None
)

# pdf = 'men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------GPA-000100--.pdf'
# men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------GPA-000200--.pdf
# pdf = 'men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------SFNL000100--.pdf'
# pdf = 'men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------QFNL000100--.pdf'
pdf = 'men/pdf/OG2020-_VVO_C83_VVOMTEAM6-------------FNL-000100--.pdf'
# men/pdf\OG2020-_VVO_C83_VVOMTEAM6-------------FNL-000100--.pdf

# ページ単位でテキスト変換
with open(pdf, "rb") as fp:
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        
# 出力結果の表示
outfp.seek(0)
text = outfp.readlines()
# print(text)
gameList = []
for t in text:
#     print(t)
    if 'vs' in t:
        team = t.replace('\n', '').replace(' ', '')
        print(team)
    if 'Round' in t or 'final' in t or 'Medal Match' in t:
        game = t.replace('\n', '').replace(' ', '')
        gameList.append(game)
print(team, gameList)
csv_name = '{}/stats/{}-{}.csv'.format(sex, gameList[0], team)
print(csv_name)

FRAvsROC
FRAvsROC
FRAvsROC
FRAvsROC
FRAvsROC ['GoldMedalMatch', 'GoldMedalMatch', 'GoldMedalMatch', 'GoldMedalMatch']
men/stats/GoldMedalMatch-FRAvsROC.csv


In [154]:
print(text)

['Match 35\n', '\n', 'Match Duration: 1h 33min\n', '\n', 'Ariake Arena\n', '有明アリーナ\n', "Arena d'Ariake\n", '\n', 'THU 5 AUG 2021\n', 'Start Time: 21:00\n', 'End Time:  22:33\n', '\n', 'Volleyball\n', ' / Volleyball\n', '\n', 'バレーボール\n', '\n', 'Men\n', ' / Hommes\n', '\n', '男子\n', '\n', 'Semifinals\n', ' / Demi-finales\n', '\n', '準決勝\n', '\n', 'Match Statistics\n', '試合のスタッツ / Statistiques du match\n', 'FRA  vs  ARG\n', '\n', 'Team\n', 'FRA\n', 'ARG\n', '\n', 'Sets\n', '3\n', '0\n', 'Playing Time\n', '\n', '3\n', '\n', '1\n', '25\n', '22\n', '27min\n', '\n', '0\n', '2\n', '25\n', '19\n', '29min\n', '\n', '3\n', '25\n', '22\n', '31min\n', '\n', 'Total\n', '75\n', '63\n', '1h 27min\n', '\n', '1st Referee:  MURANAKA Shin (JPN)\n', '\n', '2nd Referee:  LIU Jiang (CHN)\n', '\n', 'Challenge Referee: BOUDAYA Taoufik (TUN)\n', '\n', 'Scoring\n', 'Attack\n', '\n', 'Rk\n', '\n', 'No. Name\n', '\n', 'Points\n', '\n', 'Errors\n', '\n', 'In-Play\n', '\n', 'FRA - France\n', '\n', 'Team Totals\n', '\n'

In [972]:
sex = 'men'
dfs[i].to_csv('{}/stats/{}'.format(sex, csv_name))

In [27]:
import shutil
pdf = glob.glob('*VVOW*.pdf')
sex = 'women'
print(len(pdf))
# pdf
new = sex + '/pdf'
for p in pdf:
#     print(p)
    new_path = shutil.move(p, new)

# #     print(new_path)

152


In [28]:
pdf = glob.glob('women/pdf/*VVO*.pdf')
len(pdf)

152

In [1002]:
dfs[1]

,TotalRk No.,Name,Points,Errors,In-Play,Attempts,%
0,7,CONTE Facundo,19.0,6,21,46,28.26
1,12,LIMA Bruno,11.0,7,7,25,16.00
2,13,PALACIOS Ezequiel,10.0,6,10,26,15.38
3,11,SOLE Sebastian,9.0,1,4,14,NaN
4,8,LOSER Agustin,7.0,NaN,5,12,NaN
5,15,de CECCO Luciano,1.0,NaN,2,3,NaN
6,2,PEREYRA Federico,1.0,NaN,,1,NaN
7,1,SANCHEZ Matias,NaN,,,,NaN
8,6,POGLAJEN Cristian,NaN,,,,NaN
9,9,DANANI Santiago,NaN,,,,NaN


In [853]:
pdf_path = 'OG2020-_VVO_C69_VVOMTEAM6-------------FNL-000200--.pdf'
dfs = tabula.read_pdf(pdf_path, stream=True, pages='all')
print(len(dfs))
dfs[1].interpolate()

34


,Rally,Unnamed: 0,Unnamed: 1,ARG - Argentina,Unnamed: 2,Unnamed: 3,Lead,Score,Lead.1,Unnamed: 4,Unnamed: 5,Unnamed: 6,BRA - Brazil,Unnamed: 7
0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve,18.000000,LUCARELLI,In-play
1,1.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102 km/h,9.500000,NaN,NaN
2,1.666667,Attack,15.000000,de CECCO,Point,+,1.000000,1 - 0,NaN,NaN,Dig,1.000000,BRUNO,Error
3,2.000000,Serve,15.000000,de CECCO,In-play,NaN,1.000000,NaN,NaN,NaN,NaN,3.333333,NaN,NaN
4,2.333333,94 km/h,11.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,5.666667,NaN,NaN
5,2.666667,Block,7.000000,CONTE,Error,NaN,1.000000,1 - 1,NaN,+,Attack,8.000000,WALLACE,Point
6,3.000000,NaN,7.333333,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,Serve,16.000000,LUCAS,In-play
7,3.333333,NaN,7.666667,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,106 km/h,16.000000,NaN,NaN
8,3.666667,Block,8.000000,LOSER,Error,NaN,1.000000,1 - 2,1.0,+,Attack,16.000000,LUCAS,Point
9,4.000000,NaN,7.666667,NaN,NaN,NaN,1.000000,NaN,1.0,NaN,Serve,16.000000,LUCAS,In-play


In [873]:
dfList = []
for page in range(1,len(dfs)):
    df = dfs[page].copy()
#     print(page)
#     print(page, len(df.columns), df.columns)
    if len(df.columns) < 10:
        continue
#     print(df.columns)
    
#     for index in df.columns:
    df = df.rename(columns={'Lead':'Lead team1', 'Lead.1':'Lead team2'})
    if df.loc[0, 'Lead team1'] is np.nan:
        df.loc[0, 'Lead team1'] = 0
    if df.loc[0, 'Lead team2'] is np.nan:
        df.loc[0, 'Lead team2'] = 0
    df = df.dropna(how='all', axis=1)
    colList = ['Rally', 'Play team1', 'Plyaer No. team1', 'Player Name team1', 'result team1', 
               'plus/continue team1', 'Lead team1', 'Score', 'Lead team2', 'plus/continue team2', 'Play team2', 
               'Player NO. team2', 'Player Name team2', 'result team2']
    df.rename(columns=dict(zip(df.columns, colList)), inplace=True)
    
#     print(df.head(5))
    dfList.append(df)
df_all = pd.concat(dfList).reset_index().drop(columns='index')
df_all.loc[0, 'Score'] = '0 - 0'
df_all.loc[0, 'Lead team1'] = 0
df_all.loc[0, 'Lead team2'] = 0
# print(df.loc['Score', 0])
df_all[['Score', 'Lead team1', 'Lead team2']] = df_all[['Score', 'Lead team1', 'Lead team2']].fillna(method='ffill')
n = 30
df_all[n:n+50]
# df_all

,Rally,Play team1,Plyaer No. team1,Player Name team1,result team1,plus/continue team1,Lead team1,Score,Lead team2,plus/continue team2,Play team2,Player NO. team2,Player Name team2,result team2
30,NaN,Block,11.0,SOLE,Point,+,1.0,6 - 5,1.0,NaN,Attack,9.0,LEAL,Error
31,12.0,Serve,13.0,PALACIOS,In-play,NaN,1.0,6 - 5,1.0,NaN,NaN,NaN,NaN,NaN
32,NaN,92 km/h,NaN,NaN,NaN,NaN,1.0,6 - 5,1.0,NaN,NaN,NaN,NaN,NaN
33,NaN,Block,15.0,de CECCO,Point,+,2.0,7 - 5,1.0,NaN,Attack,18.0,LUCARELLI,Error
34,13.0,Serve,13.0,PALACIOS,Error,NaN,2.0,7 - 5,1.0,NaN,NaN,NaN,NaN,NaN
35,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7 - 6,1.0,+,NaN,NaN,NaN,Opp. error
36,14.0,NaN,NaN,NaN,NaN,NaN,1.0,7 - 6,1.0,NaN,Serve,1.0,BRUNO,In-play
37,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7 - 6,1.0,NaN,114 km/h,NaN,NaN,NaN
38,NaN,Dig,9.0,DANANI,Error,NaN,1.0,7 - 7,1.0,+,Attack,16.0,LUCAS,Point
39,15.0,NaN,NaN,NaN,NaN,NaN,1.0,7 - 7,1.0,NaN,Serve,1.0,BRUNO,In-play


In [774]:
print(list(df.columns))

['Rally', 'Unnamed: 0', 'Unnamed: 1', 'ARG - Argentina', 'Unnamed: 2', 'Unnamed: 3', 'Lead', 'Score', 'Lead.1', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'BRA - Brazil', 'Unnamed: 7', 0]


In [865]:
dfs[27].head()

,Rally,ARG - Argentina,Lead,Score,Lead.1,BRA - Brazil


In [836]:
dfs[3].head()

,Rally,Unnamed: 0,Unnamed: 1,Unnamed: 2,ARG - Argentina,Unnamed: 3,Unnamed: 4,Lead,Score,Lead.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,BRA - Brazil,Unnamed: 8
0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve,18.0,LUCARELLI,In-play
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115 km/h,NaN,NaN,NaN
2,NaN,Attack,12.0,LIMA,NaN,Error,NaN,NaN,9 - 9,NaN,+,NaN,NaN,NaN,Opp. error
3,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve,18.0,LUCARELLI,Error
4,NaN,NaN,NaN,NaN,NaN,Opp. error,+,1.0,10 - 9,NaN,NaN,NaN,NaN,NaN,NaN


In [838]:
dfs[5].head()

,Rally,Unnamed: 0,Unnamed: 1,ARG - Argentina,Unnamed: 2,Unnamed: 3,Lead,Score,Lead.1,Unnamed: 4,Unnamed: 5,Unnamed: 6,BRA - Brazil,Unnamed: 7
0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve,18.0,LUCARELLI,In-play
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110 km/h,NaN,NaN,NaN
2,NaN,Attack,7.0,CONTE,Point,+,4.0,19 - 15,NaN,NaN,Dig,17.0,THALES,Error
3,35.0,Serve,15.0,de CECCO,In-play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,97 km/h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### C73: 試合結果

In [647]:
l = glob.glob('OG2020-_VVO_C73*')
for pdf_path in l:
    print(pdf_path)
    dfs = tabula.read_pdf(pdf_path, stream=True, pages='all')
#     print(len(dfs))
    arrangeList = [14,11,9,6,2,0]
    df = dfs[1].iloc[3:-2,:].copy()
    d = df.drop(df.columns[arrangeList], axis=1)
    e = df[df.columns[arrangeList]]
    num = 5
    colnum = 4
    for index in arrangeList:
    #     print(index)
        item = pd.DataFrame()
        if index in [0, 9]:
            pre_item = df.iloc[:,index].str.replace('\(c\)', '', regex=True)
            item['No.'] = pre_item.str[:2]
            item['Name'] = pre_item.str[2:-2]
            item['Pos.'] = pre_item.str[-2:]
        else:
            colname = ''
            item = df.iloc[:,index].str.replace('Points by Set', 'Points bySet')
            item = item.str.split(expand=True)
            for col in range(len(item.columns)):
                colname = '{0}{1}'.format(str(colnum), str(colname))
    #             print(colname)
                item = item.rename(columns={col:colname})

            colnum -= 1

    #     print(item.head())
        loc = index - num
    #     print(loc)
        d = pd.concat([d.iloc[:,:loc], item, d.iloc[:,loc:]], axis=1)
        num -= 1

    d = d.rename(columns={'Unnamed: 0':'Lineup1', '1':'Lineup2', '11':'Lineup3', '111':'Lineup4',
                          'Unnamed: 2':'Lineup5', 'Unnamed: 3':'Points by Set1', 'Unnamed: 4':'Points by Set2',
                          '2':'Points by Set3', '22':'Points by Set4', 'Unnamed: 6':'Points by Set5', 
                          'Unnamed: 7':'Pts', 'Unnamed: 8':'Lineup1', '3':'Lineup2', '33':'Lineup3', 
                          '333':'Lineup4', 'Unnamed: 10':'Lineup5', 'Unnamed: 11':'Points by Set1', 
                          '4':'Points by Set2', '44':'Points by Set3', 'Unnamed: 13':'Points by Set4',
                          'Unnamed: 14':'Points by Set5', 'Unnamed: 15':'Pts'
                         })

    # check column!
    # print(len(d.columns))
    # print(d.dtypes)
    team1 = d.iloc[:,:14]
    team2 = d.iloc[:,14:]
    # team1

    path = pdf_path.replace('.pdf', '')
    team1.to_csv('men/{}-team1.csv'.format(path), index=False)
    team2.to_csv('men/{}-team2.csv'.format(path), index=False)

OG2020-_VVO_C73_VVOMTEAM6-------------FNL-000100--.pdf
OG2020-_VVO_C73_VVOMTEAM6-------------FNL-000200--.pdf
OG2020-_VVO_C73_VVOMTEAM6-------------GPA-000100--.pdf
OG2020-_VVO_C73_VVOMTEAM6-------------GPA-000200--.pdf


AttributeError: Can only use .str accessor with string values!

In [728]:
pdf_path = 'OG2020-_VVO_C73_VVOMTEAM6-------------GPA-000200--.pdf'
dfs = tabula.read_pdf(pdf_path, stream=True, pages='all')
# print(len(dfs))

# arrangeList = [14,11,9,6,2,0]
df = dfs[1].iloc[3:-2,:].copy()
arrangeDict = dict()
df_index = dfs[1].iloc[0, :]
df_index2 = dfs[1].iloc[1, :]
for index in range(len(df_index)):
    print(df_index[index])
    if df_index[index]=='Line-up' or df_index[index]=='Points by Set':
#         print(index)
        arrangeDict[index] = 1
    if df_index2[index]=='No. Name Pos.':
        arrangeDict[index] = 2        
# print(arrangeDict)
arrangeList = sorted(arrangeDict.items(), reverse = True)
print(arrangeList)

arrangeL = [x[0] for x in arrangeList]
print(arrangeL)

d = df.drop(df.columns[arrangeL], axis=1)
e = df[df.columns[arrangeL]]
num = 5
colnum = 4
for index in arrangeList:
#     print(index[0])
    item = pd.DataFrame()
    print(df.iloc[:,index[0]])
    if index[1] == 2:
        pre_item = df.iloc[:,index[0]].str.replace('\(c\)', '', regex=True)
        item['No.'] = pre_item.str[:2]
        item['Name'] = pre_item.str[2:-2]
        item['Pos.'] = pre_item.str[-2:]
    else:
        colname = ''
        item = df.iloc[:,index[0]]
        item = item.str.split(expand=True)
        for col in range(len(item.columns)):
            colname = '{0}{1}'.format(str(colnum), str(colname))
#             print(colname)
            item = item.rename(columns={col:colname})
            
        colnum -= 1
        
#     print(item.head())
    loc = index[0] - num
#     print(loc)
    d = pd.concat([d.iloc[:,:loc], item, d.iloc[:,loc:]], axis=1)
    num -= 1

# d = d.rename(columns={'Unnamed: 0':'Lineup1', '1':'Lineup2', '11':'Lineup3', '111':'Lineup4',
#                       'Unnamed: 2':'Lineup5', 'Unnamed: 3':'Points by Set1', 'Unnamed: 4':'Points by Set2',
#                       '2':'Points by Set3', '22':'Points by Set4', 'Unnamed: 6':'Points by Set5', 
#                       'Unnamed: 7':'Pts', 'Unnamed: 8':'Lineup1', '3':'Lineup2', '33':'Lineup3', 
#                       '333':'Lineup4', 'Unnamed: 10':'Lineup5', 'Unnamed: 11':'Points by Set1', 
#                       '4':'Points by Set2', '44':'Points by Set3', 'Unnamed: 13':'Points by Set4',
#                       'Unnamed: 14':'Points by Set5', 'Unnamed: 15':'Pts'
#                      })

# check column!
# print(len(d.columns))
# print(d.dtypes)
team1 = d.iloc[:,:14]
team2 = d.iloc[:,14:]
# team1

path = pdf_path.replace('.pdf', '')
team1.to_csv('men/{}-team1.csv'.format(path), index=False)
team2.to_csv('men/{}-team2.csv'.format(path), index=False)

nan
nan
Line-up
nan
nan
Points by Set
nan
nan
nan
nan
nan
Line-up
nan
nan
nan
Points by Set
nan
nan
[(15, 1), (11, 1), (9, 2), (5, 1), (2, 1), (0, 2)]
[15, 11, 9, 5, 2, 0]
3     NaN
4       1
5     NaN
6     3 2
7     NaN
8       3
9     NaN
10      4
11    NaN
12    1 1
13    2 2
14    2 4
Name: Unnamed: 13, dtype: object
3     6 6
4      17
5     L L
6     3 3
7      11
8     5 5
9     1 1
10    2 2
11    NaN
12    5 5
13    1 1
14    4 4
Name: Unnamed: 9, dtype: object
3       1 ARMANDO S
4     3 FERNANDO OH
5          4 MATA L
6       5 EMERSON O
7       6 ORAMAS MB
8     7 VALENCIA MB
9     9 CHEMA (c) S
10      11 VERDI MB
11     14 CANELO OH
12       15 MAITA O
13     17 FAYOLA OH
14    19 WILLNER OH
Name: VEN - Venezuela, dtype: object
3       4
4     3 4
5     NaN
6     1 2
7     4 2
8     NaN
9     6 4
10      1
11    NaN
12    NaN
13    NaN
14    4 2
Name: Unnamed: 4, dtype: object
3     11 11
4       1 6
5     12 12
6       4 3
7       5 4
8       2 1
9       3 2
10        

In [731]:
team1

,No.,Name,Pos.,Unnamed: 0,1,11,Unnamed: 2,Unnamed: 3,2,22,Unnamed: 5,Unnamed: 6,Unnamed: 7,No.
3,1,SHIMIZU,O,11,11,11,NaN,NaN,4,None,NaN,NaN,4,1
4,2,ONODERA,MB,6,1,6,NaN,2.0,3,4,NaN,NaN,9,3
5,3,FUJII,S,12,12,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
6,6,YAMAUCHI,MB,3,4,3,NaN,4.0,1,2,NaN,NaN,7,5
7,11,NISHIDA,O,4,5,4,NaN,2.0,4,2,NaN,NaN,8,6
8,12,SEKITA,S,1,2,1,NaN,1.0,NaN,NaN,NaN,NaN,1,7
9,14,ISHIKAWA,OH,2,3,2,NaN,5.0,6,4,NaN,NaN,15,9
10,15,LEE,MB,NaN,6,None,NaN,NaN,1,None,NaN,NaN,1,11
11,17,TAKANASHI,OH,NaN,14,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
12,19,OTSUKA,OH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15


In [649]:
dfs[1]

,JPN - Japan,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,VEN - Venezuela,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,Line-up,NaN,NaN,Points by Set,NaN,NaN,NaN,NaN,NaN,Line-up,NaN,NaN,NaN,Points by Set,NaN,NaN
1,No. Name Pos.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pts,No. Name Pos.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pts
2,NaN,1,2 3 4,5.0,1.0,2 3,4.0,5.0,NaN,NaN,1,2 3 4,NaN,5.0,1.0,2 3 4,5.0,NaN
3,1 SHIMIZU O,11,11 11,NaN,NaN,4,NaN,NaN,4,1 ARMANDO S,6,6 6,NaN,NaN,NaN,NaN,NaN,NaN
4,2 ONODERA MB,6,1 6,NaN,2.0,3 4,NaN,NaN,9,3 FERNANDO OH,NaN,17,NaN,NaN,NaN,1,NaN,1
5,3 FUJII S,12,12 12,NaN,NaN,NaN,NaN,NaN,NaN,4 MATA L,L,L L,NaN,NaN,NaN,NaN,NaN,NaN
6,6 YAMAUCHI MB,3,4 3,NaN,4.0,1 2,NaN,NaN,7,5 EMERSON O,3,3 3,NaN,NaN,4.0,3 2,NaN,9
7,11 NISHIDA O,4,5 4,NaN,2.0,4 2,NaN,NaN,8,6 ORAMAS MB,NaN,11,NaN,NaN,NaN,NaN,NaN,NaN
8,12 SEKITA S,1,2 1,NaN,1.0,NaN,NaN,NaN,1,7 VALENCIA MB,5,5 5,NaN,NaN,4.0,3,NaN,7
9,14 ISHIKAWA (c) OH,2,3 2,NaN,5.0,6 4,NaN,NaN,15,9 CHEMA (c) S,1,1 1,NaN,NaN,NaN,NaN,NaN,NaN


In [690]:
pdf_path = 'OG2020-_VVO_C73_VVOMTEAM6-------------GPA-000200--.pdf'
dfs = tabula.read_pdf(pdf_path, stream=True, pages='all')
df = dfs[1]

In [716]:
arrangeDict = dict()
df_index = df.iloc[0, :]
df_index2 = df.iloc[1, :]
for index in range(len(df_index)):
#     print(df_index[index])
    if df_index[index]=='Line-up' or df_index[index]=='Points by Set':
#         print(index)
        arrangeDict[index] = 1
    if df_index2[index]=='No. Name Pos.':
        arrangeDict[index] = 2
        
print(arrangeDict)

{0: 2, 2: 1, 5: 1, 9: 2, 11: 1, 15: 1}


In [717]:
arrangeList = sorted(arrangeDict.items(), reverse = True)
print(arrangeList)

[(15, 1), (11, 1), (9, 2), (5, 1), (2, 1), (0, 2)]


In [688]:
print(df_index)

ITA - Italy     No. Name Pos.
Unnamed: 0                NaN
Unnamed: 1                NaN
Unnamed: 2                NaN
Unnamed: 3                NaN
Unnamed: 4                NaN
Unnamed: 5                NaN
Unnamed: 6                NaN
Unnamed: 7                Pts
CAN - Canada    No. Name Pos.
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               NaN
Unnamed: 13               NaN
Unnamed: 14               NaN
Unnamed: 15               Pts
Name: 1, dtype: object


In [644]:
team1

,No.,Name,Pos.,Lineup1,Lineup2,Lineup3,Lineup4,Lineup5,Points by Set1,Points by Set2,Points by Set3,Points by Set4,Points by Set5,Pts
3,1,SANCHEZ,S,12,12,None,None,12,NaN,NaN,NaN,NaN,NaN,NaN
4,2,PEREYRA,O,NaN,12,None,None,NaN,NaN,NaN,1,None,NaN,1
5,6,POGLAJEN,OH,13,13,13,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,CONTE,OH,3,3,4,4,4,9.0,1,4,3,4.0,21
7,8,LOSER,MB,4,4,5,2,2,4.0,2,3,4,1.0,14
8,9,DANANI,L,L,L,L,L,L,NaN,NaN,NaN,NaN,NaN,NaN
9,11,SOLE,MB,1,1,2,5,5,2.0,3,3,2,2.0,12
10,12,LIMA,O,5,5,6,3,3,3.0,3,2,4,1.0,13
11,13,PALACIOS,OH,6,6,1,1,1,2.0,3,3,5,NaN,13
12,15,de CECCO,S,2,2,3,6,6,2.0,NaN,NaN,NaN,1.0,3


In [606]:
team2

,No.,Name,Pos.,Lineup1,Lineup2,Lineup3,Lineup4,Lineup5,Points by Set1,Points by Set2,Points by Set3,Points by Set4,Points by Set5,Pts
3,1,CHINENYEZE,MB,2,1,2,4,1,NaN,1,2,3,1.0,7
4,2,GREBENNIKOV,L,L,L,L,L,L,NaN,NaN,NaN,NaN,NaN,NaN
5,4,PATRY,O,3,2,3,5,2,4.0,3,4,2,2.0,15
6,6,TONIUTTI,S,NaN,4,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,TILLIE,OH,NaN,1,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,NGAPETH,OH,1,6,1,3,6,8.0,9,4,2,3.0,26
9,11,BRIZARD,S,6,5,6,2,5,3.0,NaN,1,None,1.0,5
10,12,BOYER,O,NaN,11,17,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,14,le GOFF,MB,5,4,5,1,4,2.0,5,1,1,NaN,9
12,16,BULTOR,MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [420]:
# for i in e:
#     print(df[i])
df.iloc[:,0].str[2:-2]

0              NaN
1        . Name Po
2              NaN
3      CHINENYEZE 
4      GREBENNIKOV
5            PATRY
6     TONIUTTI (c)
7          TILLIE 
8         NGAPETH 
9          BRIZARD
10           BOYER
11        le GOFF 
12         BULTOR 
13       CLEVENOT 
14         LOUATI 
Name: FRA - France, dtype: object